# Set up
* Versuch mit ImageDataGenerator

In [1]:
import numpy as np
import keras
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import Activation, Dropout
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

C:\ProgramData\Anaconda3\envs\deeplearning\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import glob
import os
from random import shuffle

In [ ]:
dest_train_cat = 'datasets/train/cat'
dest_train_dog = 'datasets/train/dog'

dest_valid_cat = 'datasets/valid/cat'
dest_valid_dog = 'datasets/valid/dog'

dest_test_cat = 'datasets/test/cat'
dest_test_dog = 'datasets/test/dog'

In [ ]:
splits = {
    'train_split': 0.8,
    'val_split': 0.1,
    'test_split': 0.1}

In [ ]:
train_split = splits['train_split']
val_split = splits['val_split']
test_split = splits['test_split']

In [ ]:
source = 'datasets/train_large/'
files = os.listdir(source)
len(files)

In [ ]:
shuffle_data = True
if shuffle_data == True:
    shuffle(files)

In [ ]:
train_addrs = files[0:int(train_split*len(files))]
valid_addrs = files[int((1 - 2 * val_split)*len(files)):int((1-val_split)*len(files))]
test_addrs = files[int((1 - test_split) * len(files)):]

In [ ]:
for address in train_addrs:
    if address.startswith('cat'):
        os.symlink(source + str(address), dest_train_cat + '/' + str(address))
    elif address.startswith('dog'):
        os.symlink(source + str(address), dest_train_dog + '/' + str(address))

In [ ]:
for address in valid_addrs:
    if address.startswith('cat'):
        os.symlink(source + str(address), dest_valid_cat + '/' + str(address))
    elif address.startswith('dog'):
        os.symlink(source + str(address), dest_valid_dog + '/' + str(address))

In [ ]:
for address in test_addrs:
    if address.startswith('cat'):
        os.symlink(source + str(address), dest_test_cat + '/' + str(address))
    elif address.startswith('dog'):
        os.symlink(source + str(address), dest_test_dog + '/' + str(address))

In [ ]:
train_path = 'C:/Users/JansPC/AnacondaProjects/Weinseminar/wineModel/datasets//train'
valid_path = 'C:/Users/JansPC/AnacondaProjects/Weinseminar/wineModel/datasets//valid'
test_path = 'C:/Users/JansPC/AnacondaProjects/Weinseminar/wineModel/datasets//test'

In [5]:
train_path = 'C:\\Users\\JansPC\\AnacondaProjects\\Weinseminar\\wineModel\\datasets\\train'
valid_path = 'C:\\Users\\JansPC\\AnacondaProjects\\Weinseminar\\wineModel\\datasets\\valid'
test_path = 'C:\\Users\\JansPC\\AnacondaProjects\\Weinseminar\\wineModel\\datasets\\test'

In [3]:
train_path = 'datasets\train'
valid_path = 'datasets\valid'
test_path = 'datasets\test'

In [6]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, 
                                                         target_size=(300,150), 
                                                         class_mode = "binary",
                                                         batch_size=16,
                                                         follow_links=True)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, 
                                                         target_size=(300,150), 
                                                         class_mode = "binary", 
                                                         batch_size=16,
                                                         follow_links=True)
test_batches = ImageDataGenerator().flow_from_directory(test_path, 
                                                        target_size=(300,150), 
                                                        class_mode = "binary",
                                                        batch_size=5,
                                                        follow_links=True)

Found 20000 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [7]:
# plots images with labels within jupyter notebook
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [7]:
imgs, labels = next(train_batches)

OSError: [Errno 22] Invalid argument: 'C:\\Users\\JansPC\\AnacondaProjects\\Weinseminar\\wineModel\\datasets\\train\\cat\\cat.3889.jpg'

In [ ]:
plots(imgs, titles=labels)

In [ ]:
tensor_logger = TensorBoard(
    log_dir='./logs', 
    histogram_freq=0, 
    batch_size=16, 
    write_graph=True, 
    write_grads=False, 
    write_images=False, 
    embeddings_freq=0, 
    embeddings_layer_names=None, 
    embeddings_metadata=None)

# Build and train CNN

In [ ]:
model = Sequential([
        Conv2D(24, (11, 11), strides = (3, 3), activation='relu', input_shape=(300,150,3), name='Conv1'),
        BatchNormalization(axis = 3, name = 'bn1'),
        Activation('relu'),
        Dropout(0.5),

        MaxPooling2D((3, 3), strides=(2,2), name='max_pool1'),   

        Conv2D(48, (7, 7), strides = (1, 1), name = 'conv2'),
        BatchNormalization(axis = 3, name = 'bn2'),
        Activation('relu'),
        Dropout(0.5),
    
        MaxPooling2D((3, 3), strides=(2,2), name='max_pool2'),
    
        Conv2D(96, (3, 3), strides = (1, 1), name = 'conv3'),
        BatchNormalization(axis = 3, name = 'bn3'),
        Activation('relu'),
        Dropout(0.5),

        MaxPooling2D((3, 3), strides=(2,2), name='max_pool3'),

        Flatten(),
        Dense(800, activation='relu', name='fc1'),
        Dense(800, activation='relu', name='fc2'),
        Dense(1, activation='sigmoid', name='fc3'),
    ])

In [ ]:
model.compile(Adam(lr=.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit_generator(train_batches, 
                    steps_per_epoch=1000, 
                    validation_data=valid_batches, 
                    validation_steps=100, 
                    epochs=5, 
                    verbose=2, 
                    callbacks= None) #[tensor_logger])

# Predict

In [ ]:
test_imgs, test_labels = next(test_batches)
plots(test_imgs, titles=test_labels)

In [ ]:
test_labels = test_labels[:]
test_labels

In [ ]:
predictions = model.predict_generator(test_batches, steps=1, verbose=0)

In [ ]:
predictions

In [ ]:
cm = confusion_matrix(test_labels, predictions)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm_plot_labels = ['cat','dog']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

## Build Fine-tuned VGG16 model

In [ ]:
vgg16_model = keras.applications.vgg16.VGG16()

In [ ]:
vgg16_model.summary()

In [ ]:
type(vgg16_model)

In [ ]:
model = Sequential()
for layer in vgg16_model.layers:
    model.add(layer)

In [ ]:
model.summary()

In [ ]:
model.layers.pop()

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
    layer.trainable = False

In [ ]:
model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

## Train the fine-tuned VGG16 model

In [ ]:
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_batches, steps_per_epoch=4, 
                    validation_data=valid_batches, validation_steps=4, epochs=5, verbose=2)

In [ ]:
#old model results
#model.fit_generator(train_batches, steps_per_epoch=4, 
#                    validation_data=valid_batches, validation_steps=4, epochs=5, verbose=2)

## Predict using fine-tuned VGG16 model

In [ ]:
test_imgs, test_labels = next(test_batches)
plots(test_imgs, titles=test_labels)

In [ ]:
test_labels = test_labels[:,0]
test_labels

In [ ]:
predictions = model.predict_generator(test_batches, steps=1, verbose=0)

In [ ]:
cm = confusion_matrix(test_labels, np.round(predictions[:,0]))

In [ ]:
cm_plot_labels = ['cat','dog']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

In [ ]:
#old model results
cm_plot_labels = ['cat','dog']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')